In [ ]:
!pip install nudged

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import json
import psutil
import random
import os
import time
import sys
import math
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm
from contextlib import contextmanager
import matplotlib.pylab as plt
import shapely
import matplotlib
from shapely.geometry import shape, GeometryCollection, Polygon, MultiPolygon
from shapely.affinity import affine_transform
from PIL import Image, ImageOps
import nudged
from skimage.morphology import convex_hull_image
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pickle
from shapely.geometry import Point
from scipy.spatial.distance import cdist

In [ ]:
#Provided GitHub functions
!cp -r ../input/indoor-location-competition-20-git/* ./
import io_f
import visualize_f 
import compute_f
import main  

# FUNCTIONS

* ### Additional

In [ ]:
floors_key_assign = {'1F':0, '2F':1, '3F':2, '4F':3, '5F':4, '6F':5, '7F':6, '8F':7, '9F':8, 
          'B':-1, 'B1':-1,'B2':-2, 'B3':-3, 'BF':-1, 'BM':-1, 
          'F1':0, 'F2':1, 'F3':2, 'F4':3, 'F5':4, 'F6':5,'F7':6, 'F8':7, 'F9':8, 'F10':9,
          'G':0, 'LG1':-1, 'LG2':-2, 
          "L1":0,"L2":1,"L3":2,"L4":3,"L5":4,"L6":5,"L7":6,"L8":7,"L9":8,"L10":9,'L11':10, 
          'LM':0, 'M':0, 'P1':-1, 'P2':-2,
          "地下一层":-1,"一层":0,"二层":1,"三层":2,"四层":3}
def get_site_floor_from_id(site, floor_id):
    site_p = "../input/indoor-location-navigation/train/"+site
    floor_p = "/*"
    site_floor_names = []
    floors_files = sorted(glob.glob(site_p + floor_p))
    for f, floor in enumerate(floors_files):
        floor_name = floor.split('/')[-1]
        site_floor_names.append(floor_name)
    for key_f in floors_key_assign:
        if key_f in site_floor_names:
            if floors_key_assign[key_f] == floor_id:
                return key_f
    return floor_id

In [ ]:
def highlight_dif(x):
    r = 'red'
    g = 'green'

    m1 = x['f'] != x['pred_f']
    m2 = x['f'] == x['pred_f']
    
    m3 = x['x'] != x['pred_x']
    m4 = x['x'] == x['pred_x']
    
    m5 = x['y'] != x['pred_y']
    m6 = x['y'] == x['pred_y']

    df1 = pd.DataFrame('color: ', index=x.index, columns=x.columns)
    #rewrite values by boolean masks
    df1['pred_f'] = np.where(m1, 'color: {}'.format(r), df1['pred_f'])
    df1['pred_f'] = np.where(m2, 'color: {}'.format(g), df1['pred_f'])
    
    df1['pred_x'] = np.where(m3, 'color: {}'.format(r), df1['pred_x'])
    df1['pred_x'] = np.where(m4, 'color: {}'.format(g), df1['pred_x'])
    
    df1['pred_y'] = np.where(m5, 'color: {}'.format(r), df1['pred_y'])
    df1['pred_y'] = np.where(m6, 'color: {}'.format(g), df1['pred_y'])
    return df1

* ### Plot on Map

In [ ]:
def plot_floor(
    site,
    floorNo,
    paths_wp_df=None,
    display_corridors=True,
    display_map=True,
    base="../input/indoor-location-navigation",
    base_pol = "../input/indoor-location-navigation-scaled-geojson/scaled_geojson",
):
    map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]
    floor_img = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    
    file = f"{base_pol}/{site}/{map_floor}/shapely_geometry.pkl"
    with open(file, 'rb') as f:
        geometry = pickle.load(f)
    
    m_opacity=0.8
    colorbar_title="RSSI"
    #Display
    fig = go.Figure()
    if display_corridors:
        for coord in extract_coords_from_polygon(geometry):
            x, y = coord
            fig.add_trace(
                go.Scattergl(
                    x=x,
                    y=y,
                    opacity=0.4,
                    text=None,
                    marker_color='orange',
                    hoverinfo='skip',
                    showlegend=False
                ))
    if paths_wp_df is not None:
        fig.add_trace(
            go.Scatter(x=paths_wp_df.x,
                       y=paths_wp_df.y,
                       mode='markers',
                       marker=dict(size=20,
                                   color=paths_wp_df.rssi,
                                   colorbar=dict(title=colorbar_title),
                                   colorscale="Rainbow"),
                       text=paths_wp_df.rssi,
                       opacity=0.7,
                        showlegend=False))
    
    if display_map:
        floor_plan = Image.open(floor_img)
        fig.update_layout(images=[
            go.layout.Image(
                source=floor_plan,
                xref="x",
                yref="y",
                x=0,
                y=height_meter,
                sizex=width_meter,
                sizey=height_meter,
                sizing="contain",
                opacity=0.5,
                layer="below",
            )
        ])
    

    
    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(coloraxis_colorbar=dict(title="RSSI"))
    fig.update_layout(
        title=go.layout.Title(
            text="Site: %s Floor: %s" % (site, map_floor),
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=700,
        height=200 + 700 * height_meter / width_meter,
        template="plotly_white",
    )

    return fig

    

* ### Generate Grid from Map

In [ ]:
def generate_grid_from_map(stite_check, floor_check, step = 2):
    print ("\033[A                                                          \033[A", end="\r")
    print("Map Grid Generatig...", end="\r")
    base_pol = "../input/indoor-location-navigation-scaled-geojson/scaled_geojson"
    file = f"{base_pol}/{stite_check}/{floor_check}/shapely_geometry.pkl"
    with open(file, 'rb') as f:
        corridor = pickle.load(f)
    
    x_min, y_min, x_max, y_max = get_bounding_box(corridor)

    x_range = range(math.ceil(x_min), math.floor(x_max), step)
    y_range = range(math.ceil(y_min), math.floor(y_max), step)

    valid_coords = []

    for x in x_range:
        for y in y_range:
            if Point(x, y).within(corridor):
                valid_coords.append([x, y])

    return np.array(valid_coords)

def get_bounding_box(shapes):
    """
    To extract bounding box from Polygon
    """
    x_min = 10000
    y_min = 10000
    x_max = 0
    y_max = 0
    
    if type(shapes) == Polygon:
            shapes = [shapes]
    for shape in shapes:
        x, y = shape.exterior.xy
        x_min = min(min(x), x_min)
        y_min = min(min(y), y_min)
        x_max = max(max(x), x_max)
        y_max = max(max(y), y_max)
    return x_min, y_min, x_max, y_max

def extract_coords_from_polygon(polygon):
    coords = []
    if type(polygon) == MultiPolygon:
        polygons = polygon.geoms
    else:
        polygons = [polygon]

    for polygon in polygons:
        x, y = polygon.exterior.xy
        coords.append((np.array(x), np.array(y)))
        for interior in polygon.interiors:
            x, y = interior.xy
            coords.append((np.array(x), np.array(y)))

    return coords

* ### Generate Grid from Paths

In [ ]:
def generate_grid_from_wp(site:str=None,floor:str=None, start:int=None, stop:int=None):
    site_p = "../input/indoor-location-navigation/train/*"
    floor_p = "/*"
    path_p = "/*"
    
    if (site is not None) & (floor is None):
        site_p = "../input/indoor-location-navigation/train/" + site
        print ("\033[A                                                          \033[A", end="\r")
        print("Grid Generatig Based on Paths in {} from all Floors...".format(site))

    if (site is not None) & (floor is not None):
        site_p = "../input/indoor-location-navigation/train/" + site
        floor_p = "/" + floor
        print ("\033[A                                                          \033[A", end="\r")
        print("Grid Generatig Based on Paths in {} from {}...".format(site,floor))
    
    #Go through sites
    sites = sorted(glob.glob(site_p))
    
    if (start is not None) & (stop is not None):
        sites = sites[start:stop]
    
    #Create DF of all waypoints in whole site
    main_wp_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])
    for s, site in enumerate(sites):
        #print site info
        site_id = site.split('/')[-1]
        print("Site", s+1, "/", len(sites), "({})".format(site_id), end="\n\r")

        #Create DF of all waypoints in whole site
        site_wp_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])
        total_path = 0

        #Go through floors
        floors = sorted(glob.glob(site + floor_p))  
        for f, floor in enumerate(floors):
            #print floor info
            floor_id = floor.split('/')[-1]
            print("~ Floor", f+1, "/", len(floors), "({})".format(floor_id), end="\n\r")

            #Create DF of all waypoints on each floor
            wp_floor_df = pd.DataFrame(columns = ['site', 'floor', 'x', 'y'])

            #Go through paths
            paths = sorted(glob.glob(floor + path_p))   
            for p, path in enumerate(paths):                
                total_path += len(paths)

                #print path info
                path_id = path.split('/')[-1].split('.')[0]
                print ("\033[A                                                          \033[A", end="\r")
                print("~~ Path", p+1, "/", len(paths), "({})".format(path_id), end="\r")

                #Exstract All Paths Waypoints
                path_wp = io_f.read_data_file(path).waypoint           
                for wp in path_wp:
                    new_row = {'site':site_id, 'floor':floor_id, 'x':wp[1], 'y':wp[2]}
                    wp_floor_df = wp_floor_df.append(new_row, ignore_index=True)

                #Drop duplicate waypoints
                wp_floor_df = wp_floor_df.drop_duplicates(subset=['x', 'y'], keep='last')

            #load to site wp df
            site_wp_df = site_wp_df.append(wp_floor_df, ignore_index=True)

        main_wp_df = main_wp_df.append(site_wp_df, ignore_index=True)

    site_floor_list = main_wp_df.groupby(['site', 'floor'],as_index=False).agg({'x': 'count'})    
    if (site is not None) | (floor is not None):
        return main_wp_df, site_floor_list
    else:
        return main_wp_df

* ### Snap to Grid

In [ ]:
def snap_to_grid(wp, grid, threshold):
    print ("\033[A                                                          \033[A", end="\r")
    print("Snapping to the Grid...", end="\r")
    wp = find_closest_grid(wp, grid)
    wp['dist'] = np.sqrt((wp.x-wp.x_)**2 + (wp.y-wp.y_)**2)
    
    wp = snap_points_to_grid(wp, threshold=threshold)
    wp = wp[['site', 'floor', 'path', 'timestamp','_x_','_y_']].rename(columns={'_x_':'x', '_y_':'y'})
    return wp

def add_xy(df):
    df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
    return df
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]
def find_closest_grid(wp, grid):
    grid_df = pd.DataFrame(grid, columns = ['x','y'])
    wp = add_xy(wp)
    grid_df = add_xy(grid_df)
    ds = []
    for (site, myfloor), d in wp.groupby(['site','floor']):
        true_floor_locs = grid_df.reset_index(drop=True)
        if len(true_floor_locs) == 0:
            print(f'Skipping {site} {myfloor}')
            continue
        d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
        d['x_'] = d['matched_point'].apply(lambda x: x[0])
        d['y_'] = d['matched_point'].apply(lambda x: x[1])
        ds.append(d)

    wp = pd.concat(ds)
    
    return(wp)
def snap_points_to_grid(sub, threshold):
    """
    Snap to grid if within a threshold.
    
    x, y are the predicted points.
    x_, y_ are the closest grid points.
    _x_, _y_ are the new predictions after post processing.
    """
    sub['_x_'] = sub['x']
    sub['_y_'] = sub['y']
    sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
    sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
    return sub.copy()

* ### Training & Validation

In [ ]:
def train_way_knn(train_pdf: pd.DataFrame, n_neighbor, leaf):
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.neighbors import KNeighborsClassifier
    
    features = train_pdf.iloc[:,:-4]
    x_label = train_pdf.iloc[:,-4]
    y_label = train_pdf.iloc[:,-3]
    f_label = train_pdf.iloc[:,-2]

    x = KNeighborsRegressor(n_neighbors=n_neighbor,weights="uniform",algorithm="ball_tree",leaf_size=leaf)
    x_model = x.fit(features, x_label)
    y = KNeighborsRegressor(n_neighbors=n_neighbor,weights="uniform",algorithm="ball_tree",leaf_size=leaf)
    y_model = y.fit(features, y_label)
    f = KNeighborsClassifier(n_neighbors=n_neighbor,weights="uniform",algorithm="ball_tree",leaf_size=leaf)
    f_model = f.fit(features, f_label)

    return x_model, y_model, f_model

def model_validation(df, model_x, model_y, model_f):
    v_features = df.iloc[:,:-4]
    v_label_x = df.iloc[:,-4]
    v_label_y = df.iloc[:,-3]
    v_label_f = df.iloc[:,-2]
    
    x_pred = model_x.predict(v_features)
    y_pred = model_y.predict(v_features)
    f_pred = model_f.predict(v_features)
        
    df = df[["path",'x','y','f']].copy()
    df['pred_x'] = x_pred
    df['pred_y'] = y_pred
    df['pred_f'] = f_pred.astype(int)
    df['wp_error'] = np.sqrt(np.power(df['pred_x'] - v_label_x, 2) + np.power(df['pred_y'] - v_label_y, 2))
    
    return df.reset_index(drop=True)

* ### Load Timestamps

In [ ]:
def load_timestamps(df, site_id):
    site_p = "../input/indoor-location-navigation/train/"+site_id
    path_p = "/*/*"
    floor_paths_files = sorted(glob.glob(site_p + path_p))
    final_df = pd.DataFrame(columns=['path','timestamp','x','y','pred_x','pred_y','f','pred_f'])
    for p, path in enumerate(floor_paths_files):
        path_id = path.split('/')[-1].split('.')[0]
        print ("\033[A                                                                                                  \033[A", end="\r")
        print("~Timestamp Assign: {:03d}/{} ({})".format(p+1, len(floor_paths_files), path_id), end='\r')
        path_wp = pd.DataFrame(io_f.read_data_file(path).waypoint, columns=['timestamp', 'x', 'y']).reset_index(drop=True)
        path_df = df[df.path == path_id].reset_index(drop=True)
        #path_wp['timestamp'] = path_wp['timestamp'] - path_wp['timestamp'].min()
        if path_df.shape[0] > 0:
            #Assign most frequent floor
            #temp = path_df.groupby(['pred_f'])["pred_f"].count().reset_index(name="count")
            #pred_floor = temp.pred_f[temp.groupby('pred_f')['count'].idxmax()].values[0]
            act_floor = path_df.f.values[0]

            if path_wp.shape[0] == path_df.shape[0]:
                df2 = pd.merge(path_wp,path_df, on=['x','y'])
            else:
                temp = pd.merge(path_wp,path_df, on=['x','y'])
                temp = temp.groupby(['timestamp','pred_x','pred_y','pred_f'])["timestamp"].count().reset_index(name="count")
                temp = temp.loc[temp.groupby('timestamp')['count'].idxmax()]
                df2 = pd.merge(path_wp,temp.drop(columns=['count']), on=['timestamp'])

            df2['path'] = path_id
            df2['f'] = act_floor
            #df2['pred_f'] = pred_floor
            final_df = final_df.append(df2)
    return final_df

* ### Get All Path from Floor

In [ ]:
def get_floor_paths(stite_check,floor_check):
    print ("\033[A                                                          \033[A", end="\r")
    print("Loading Floor Paths...", end="\r")
    base = '../input/indoor-location-navigation/train'
    floor_paths_files = sorted(glob.glob("{}/{}/{}/*".format(base,stite_check,floor_check)))
    floor_paths_df = pd.DataFrame(columns = ['site', 'floor', 'path', 'timestamp', 'x', 'y'])
    for floor_paths_file in floor_paths_files:
        path_id = floor_paths_file.split('/')[-1].split('.')[0]
        #Exstract All Paths Waypoints
        path_wp = io_f.read_data_file(floor_paths_file).waypoint
        path_wp_df = pd.DataFrame(columns = ['site', 'floor', 'path', 'timestamp', 'x', 'y'])
        for wp in path_wp:
                new_row = {'site':stite_check, 'floor':floor_check, 'path':path_id, 'timestamp':wp[0], 'x':wp[1], 'y':wp[2]}
                path_wp_df = path_wp_df.append(new_row, ignore_index=True)

        floor_paths_df = floor_paths_df.append(path_wp_df, ignore_index=True)
    return (floor_paths_df)

* ### Cost Minimisation

In [ ]:
def compute_rel_positions(acce_datas, ahrs_datas, posi_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    step_positions = compute_f.correct_positions(rel_positions, posi_datas)
    return rel_positions, step_positions

def correct_path(site, path_df, alpha_mod):
    site_p = "../input/indoor-location-navigation/train/"
    path = path_df['path'].values[0]
    floor = path_df['floor'].values[0]
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    site_floor = get_site_floor_from_id(site,floor)
    
    example = io_f.read_data_file(f'{site_p}/{site}/{site_floor}/{path}.txt')
    
    rel_positions, step_positions = compute_rel_positions(example.acce, example.ahrs, path_df[['timestamp','x','y']].to_numpy())
    posi_datas = pd.DataFrame(columns=['timestamp','x', 'y'])
    posi_datas['timestamp'] = step_positions[:, 0]
    posi_datas['x'] = step_positions[:, 1]
    posi_datas['y'] = step_positions[:, 2]
    posi_datas['floor'] = floor
    posi_datas['path'] = path
    
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-alpha_mod) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags( beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)
    
    path_df['x'] = xy_star[:, 0]
    path_df['y'] = xy_star[:, 1]
    
    return posi_datas, path_df

# SCRIPTS

* ### Training Script

In [ ]:
def knn_site_train(site_to_check=None, n_split=10, ts_ass=True):
    from sklearn.model_selection import KFold
    from sklearn.model_selection import train_test_split
    N_SPLITS = n_split
    SEED = 42
    def set_seed(seed=42):
        random.seed(seed)
        os.environ["PYTHONHASHSEED"] = str(seed)
        np.random.seed(seed)

    set_seed(SEED)
    score_df = pd.DataFrame()

    feature_dir = "../input/indoor-navigation-and-location-wifi-features/wifi_features/train"
    train_files = sorted(glob.glob(os.path.join(f'{feature_dir}/*_1000_train.csv')))
    if site_to_check is not None:
        train_files = sorted(glob.glob(os.path.join(f'{feature_dir}/{site_to_check}_1000_train.csv')))
    sites_pred_eval = pd.DataFrame(columns = ['site','wp_mpe','f_acc'])
    
    for e, file in enumerate(train_files):
        total_mpe = []
        total_x_acc = []
        total_y_acc = []
        total_f_acc = []
        
        final_predictions = pd.DataFrame(columns = ['path','x','y','f','pred_x','pred_y','pred_f'])
        site_id = file.split('/')[-1].split("_")[0]
        print("Site {}/{} ({})".format(e+1, len(train_files), site_id))
        print ("\033[A                                                                                                  \033[A", end="\r")
        print("~Getting CSV...", end='\r')
        data = pd.read_csv(file, index_col=0)
        
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        print("~Folding the Dataset...", end='\r')
        for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
            print ("\033[A                                                                                                  \033[A", end="\r")
            print("~Folder {}/{} | Splitting Dataset...".format(fold+1, N_SPLITS), end='\r')
            train = data.iloc[trn_idx]
            valid = data.iloc[val_idx]

            print ("\033[A                                                                                                  \033[A", end="\r")
            print("~Folder {:02d}/{} | KNN Trtaining...".format(fold+1, N_SPLITS), end='\r')
            model_knn_x, model_knn_y, model_knn_f = train_way_knn(train, 1, 17)

            print ("\033[A                                                                                                  \033[A", end="\r")
            print("~Folder {:02d}/{} | Validating...".format(fold+1, N_SPLITS), end='\r')
            pred_knn_df = model_validation(valid, model_knn_x, model_knn_y, model_knn_f)

            print ("\033[A                                                                                                  \033[A", end="\r")
            print("~Folder {:02d}/{} | Finishing Up...".format(fold+1, N_SPLITS), end='\r')  
            knn_mpe = pred_knn_df.wp_error.mean()
            knn_x_acc = sum(pred_knn_df.x == pred_knn_df.pred_x)/pred_knn_df.shape[0]
            knn_y_acc = sum(pred_knn_df.y == pred_knn_df.pred_y)/pred_knn_df.shape[0]
            knn_f_acc = sum(pred_knn_df.f == pred_knn_df.pred_f)/pred_knn_df.shape[0]
            total_mpe.append(knn_mpe)
            total_x_acc.append(knn_x_acc)
            total_y_acc.append(knn_y_acc)
            total_f_acc.append(knn_f_acc)

            if ts_ass:
                final_predictions = final_predictions.append(pred_knn_df[['path','x','y','pred_x','pred_y','f','pred_f']], ignore_index=True)

        
        
        print ("\033[A                                                                                                  \033[A", end="\r")
        print("~DONE | WP MPE: {:.5f} | X ACC: {:.5f} | Y ACC: {:.5f} | F ACC: {:.5f}".format(
            np.mean(total_mpe), np.mean(total_x_acc), np.mean(total_y_acc), np.mean(total_f_acc)))

        new_row = {'site':site_id, 'wp_mpe':np.mean(total_mpe),'x_acc':np.mean(total_x_acc),'y_acc':np.mean(total_y_acc), 'f_acc':np.mean(total_f_acc)}
        sites_pred_eval = sites_pred_eval.append(new_row, ignore_index=True)
        
        if ts_ass:
            final_predictions = load_timestamps(final_predictions, site_id)
            final_predictions['timestamp'] = final_predictions['timestamp'].astype(int)
            final_predictions = final_predictions.sort_values(['path','timestamp']).reset_index(drop=True)
            final_predictions.to_csv('./knn_fold_predictions/{}.csv'.format(site_id),index=False)

    return sites_pred_eval


* ### Grid Generation

In [ ]:
def generate_grid_for_floor(stite_check, floor_check, step):
    #collect all waypoints on floor
    floor_paths_df = get_floor_paths(stite_check,floor_check)
    #grid from waypoints
    floor_wp_grid_df, site_floor_list = generate_grid_from_wp(stite_check, floor_check)
    floor_wp_grid = floor_wp_grid_df[['x','y']].to_numpy() #for ploting
    #generate grid based on map
    floor_map_grid = generate_grid_from_map(stite_check, floor_check, step)
    #snap to grid
    #fixed_floor_paths_df = snap_to_grid(floor_paths_df, floor_map_grid, 2)
    print ("\033[A                                                          \033[A", end="\r")
    print("Grid Generated | Site: {}, Floor: {}".format(stite_check, floor_check))
    return floor_paths_df, floor_wp_grid, floor_map_grid

# POST PROCESSING

In [ ]:
stop = stop

# RUNTIME

In [ ]:
outdir = './knn_fold_predictions'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
pred_eval = knn_site_train(site_to_check=None, n_split=10, ts_ass=True)
display(pred_eval)
print(pred_eval.wp_mpe.mean())
pred_eval.to_csv('knn_fold_predictions/knn_fold_eval.csv',index=False)

In [ ]:
site_sample_path = './knn_fold_predictions/5d27096c03f801723c31e5e0.csv'
site_to_check = site_sample_path.split('/')[-1].split('.')[0]
site_df = pd.read_csv(site_sample_path)

# RSSI ANALYSIS

In [ ]:
rm_path = '../input/indoor-navigation-and-location-wifi-features/wifi_features/train/5da138274db8ce0c98bbd3d2_1000_train.csv'
site_id = rm_path.split('/')[-1].split("_")[0]
floor = 0
floor_id = get_site_floor_from_id(site_id, floor)
rm_df = pd.read_csv(rm_path)
print(f"Site: {site_id}, Floor: {floor_id}({floor})")

In [ ]:
rm_df.shape

In [ ]:
rm_df.loc[rm_df.f==floor]

In [ ]:
path_rm_df.shape

In [ ]:
path_rm_df

In [ ]:
ap = '28ec3d3d264c01931d61ce5ec6882334c75e0496'
path_rm_df = rm_df[[ap, 'x','y','path']].loc[rm_df.f==floor]
path_rm_df = path_rm_df.rename({ap: 'rssi'}, axis='columns')
path_rm_df = path_rm_df[path_rm_df.rssi != -999]
plot_floor(site_id, floor_id,path_rm_df)

In [ ]:
import plotly.express as px
fig = px.scatter(path_rm_df, x="x", y="y", color="rssi", color_continuous_scale = ["blue","aqua","lime",'yellow', "red"])
fig.update_layout(coloraxis_colorbar=dict(title="RSSI"))
fig.show()